# Budget

## import

In [ ]:
from investment.models import Budget, FinancialManager, FinancialInstitutions,\
CashFlowType, CashFlowAccountingCategorie, CashFlowFrequency, SpendingTheme
from django.contrib.auth.models import User
import csv

In [ ]:
for u in User.objects.all():
    print(u.id)
user = User.objects.get(id=1)

In [ ]:
# Budget.objects.all().delete()

In [ ]:
import chardet
file_path = 'data/BUdget.csv'
data_list = []
with open(file_path, 'rb') as rawdata:
    result = chardet.detect(rawdata.read())
# with open(file_path, 'r', encoding='utf-8-sig') as csvfile:
with open(file_path, 'r', encoding=result['encoding']) as csvfile:
    # Create a DictReader object
    csv_reader = csv.DictReader(csvfile)
    # Iterate through each row (each row is a dictionary)
    for row in csv_reader:
        # Remove \ufe from the start of each value
        # cleaned_row = {key: value.replace('\ufe', '') for key, value in row.items()}
        # print(row)  # Print each row (dictionary)
        data_list.append(row)

In [ ]:
for data in data_list:
    b = Budget()
    for k,v in data.items():
        if k == 'amount':
            v = abs(float(v.replace(",", "").replace("(", "").replace(")", "")))
        if k == 'financial_institution':
            # print(v)
            v = FinancialInstitutions.objects.get(name=v)
        if k == 'cost_type':
            v = CashFlowType.objects.get(name=v)
        if k == 'accounting_category':
            v = CashFlowAccountingCategorie.objects.get(name=v)
            # print(v)
        if k == 'frequency':
            v = CashFlowFrequency.objects.get(name=v)
        if k == 'spending_theme':
            if v != "":
                print(v)
                v = SpendingTheme.objects.get(name=v)
            else:
                v= None
            # print(v)
            
        setattr(b, k, v)
    setattr(b, 'owner', user)
    print(b)
    # b.save()

In [ ]:
fm = FinancialManager(user=user)


## Revenus, Depenses, Epargne

### Détail

In [ ]:
def get_budget_items_by_accounting_category_dict():
    budget = Budget.objects.filter(owner=user)
    cash_flow_accountin_type_qs = CashFlowAccountingCategorie.objects.all()
    cash_flow_frequency_qs = CashFlowFrequency.objects.all()
    result_dict = {}
    for cash_flow_accountin_type in cash_flow_accountin_type_qs:
        result_dict[cash_flow_accountin_type.name] = [b for b in budget.filter(accounting_category=cash_flow_accountin_type)]
    return result_dict    

for accounting_category_name, budget_obj_list in get_budget_items_by_accounting_category_dict().items():
    print(accounting_category_name)

    for budget_obj in budget_obj_list:
        print("*******", budget_obj.spending_item_name, budget_obj.calculate_amount())

### Résumé Revenus Dépenses, epargne

In [ ]:
result_dict = {}
result_dict["Savings"] = {}
total_expenses = 0
total_revenus = 0
cash_flow_frequency_qs = CashFlowFrequency.objects.all()
for accounting_category, budget_obj_list in get_budget_items_by_accounting_category_dict().items():
    result_dict[accounting_category]={}
    for cash_flow_frequency in cash_flow_frequency_qs:
        amount = round(sum([b.calculate_amount(frequency_name=cash_flow_frequency.name) for b in budget_obj_list]), 2)
        result_dict[accounting_category][cash_flow_frequency.name] = amount
        if cash_flow_frequency.name not in result_dict["Savings"].keys():
            result_dict["Savings"][cash_flow_frequency.name]=0
        elif accounting_category == "Revenus":
            result_dict["Savings"][cash_flow_frequency.name] += amount
        elif accounting_category in ["Expenses", "Amortization"]:
            result_dict["Savings"][cash_flow_frequency.name] -= amount
            
        
        
        

print(result_dict)

## Transferts aux Institutions financieres

In [ ]:
result_dict = {}
for financial_institution in FinancialInstitutions.objects.all():
    result_dict[financial_institution.name] = {}
    for cash_flow_frequency in CashFlowFrequency.objects.all():
        result_dict[financial_institution.name][cash_flow_frequency.name] = round(sum([b.calculate_amount(frequency_name=cash_flow_frequency.name) 
                                                                                 for b in Budget.objects.filter(financial_institution=financial_institution, 
                                                                                                                accounting_category__id = 2)]), 2)
        
result_dict

In [ ]:
def get_cash_transfert_to_financial_institutions_dict():
    result_dict = {}
    for financial_institution in FinancialInstitutions.objects.all():
        result_dict[financial_institution.name] = {}
        for cash_flow_frequency in CashFlowFrequency.objects.all():
            result_dict[financial_institution.name][cash_flow_frequency.name] = round(sum([b.calculate_amount(frequency_name=cash_flow_frequency.name) 
                                                                                     for b in Budget.objects.filter(financial_institution=financial_institution, 
                                                                                                                    accounting_category__id = 2)]), 2)
    return result_dict
get_cash_transfert_to_financial_institutions_dict()

### Case Use

In [ ]:
def get_non_zero_ct_by_frquency_dict(frequency_name='Bi-Weekly'):
    cash_transfer_list = []
    for financial_institution_name, frequency_amount in get_cash_transfert_to_financial_institutions_dict().items():
        for frequency, amount in frequency_amount.items():
            if frequency == frequency_name and amount != 0:
                cash_transfer_list.append((financial_institution_name, amount))
    return cash_transfer_list
                
print(get_non_zero_ct_by_frquency_dict())

### Sunburn chart cash transfert

In [ ]:
def graph_non_zero_ct_by_frquency_dict(frequency_name='Bi-Weekly'):
    parent_labels = []
    parent_values = []
    for financial_institution_name, frequency_amount in get_cash_transfert_to_financial_institutions_dict().items():
        for frequency, amount in frequency_amount.items():
            if frequency == frequency_name and amount != 0:
                parent_labels.append(financial_institution_name)
                parent_values.append(amount)
                
    child_labels = []
    child_values = []   
    for budget_item in Budget.objects.filter(owner__id=user.id).exclude(accounting_category__id=1).order_by('financial_institution'):
        if budget_item.spending_item_name != "Paye":
            child_labels.append((budget_item.financial_institution, budget_item.spending_item_name))
            child_values.append(budget_item.calculate_amount(frequency_name=frequency_name))
            # print(budget_item.financial_institution, budget_item.spending_item_name, budget_item.calculate_amount(frequency_name=frequency_name))
            
    parents = [x[0] for x in child_labels]
    number_of_parents = len(set(parents))
    parents = [""] * number_of_parents + parents
    categories = parent_labels + [x[1] for x in child_labels]
    values = parent_values + child_values
    
    return parents, categories, values

   
        

parents, categories, values = graph_non_zero_ct_by_frquency_dict(frequency_name='Bi-Weekly')

for parent, label, value in zip(parents, categories, values):
    print(parent, label, value)

In [ ]:
financial_institution_qs = FinancialInstitutions.objects.all().order_by('name')
result_dict = {}
for financial_institution in financial_institution_qs:
    result_list = [(b.spending_item_name, b.calculate_amount(frequency_name="Bi-Weekly")) for b in 
                   Budget.objects.filter(owner=user, financial_institution=financial_institution).order_by('-amount') 
                   if b.spending_item_name != "Paye"]
    if result_list:
        result_dict[financial_institution.name] = result_list
    

print(result_dict)

## Transfert entre institution résumé

In [ ]:
financial_institution_qs = FinancialInstitutions.objects.all().order_by('name')
result_dict = {}
for financial_institution in financial_institution_qs:
    amount = round(sum([b.calculate_amount(frequency_name="Bi-Weekly") for b in 
                                                Budget.objects.filter(owner=user, financial_institution=financial_institution)]), 2)
    if amount > 0:
        result_dict[financial_institution] = amount
        
    total_amount = round(sum([b.calculate_amount(frequency_name="Bi-Weekly") for b in 
                                                Budget.objects.filter(owner=user)]), 2)

print(result_dict, total_amount)

    

In [ ]:
fm.calculate_total_monthly_spending()

## Pie chart 1

In [ ]:
data = fm.get_budget_summary_by_frequency()
labels, values = zip(*data)
print(labels, values)

In [ ]:
get_budget_items_by_accounting_category_dict()

In [ ]:

total_spending_ammount = 0
total_revenus = 0
labels = []
values = []
for accounting_category, budget_obj_list in get_budget_items_by_accounting_category_dict().items():
    if accounting_category == 'Spending':
        for budget_obj in budget_obj_list:
            amount = budget_obj.calculate_amount()
            total_spending_ammount += amount
            labels.append(budget_obj.spending_item_name)
            values.append(amount)
    if accounting_category == 'Revenus':
        for budget_obj in budget_obj_list:
            total_revenus += budget_obj.calculate_amount()
            
labels.append("Épargne")
values.append(round(total_revenus - total_spending_ammount, 2))
    
print(labels, values, total_revenus)

## Pie chart 2

## Budget detail view

In [ ]:

labels = []
values = []
SpendingTheme.objects.all()
for spendingtheme in SpendingTheme.objects.all():
    labels.append(spendingtheme.name)
    values.append(round(sum([b.calculate_amount() for b in spendingtheme.budget_set.all() if b.accounting_category.id in [2, 4]]), 2))

total_spending = sum(values)
total_revenus = sum([b.calculate_amount() for b in Budget.objects.all() if b.accounting_category.id == 1])
labels.append('savings')
values.append(round(total_revenus - total_spending, 2))
print(labels, values)

In [ ]:
categories_with_expenses = CashFlowAccountingCategorie.objects.filter(
    name='Expenses',
    budget__spending_theme__isnull=False
).annotate(
    spending_theme=F('budget__spending_theme'),
    total_amount=Sum('budget__amount')
).order_by('name', 'spending_theme')

categories_with_expenses

## Pie chart 3

In [ ]:
[b.owner for b in Budget.objects.all() if b.accounting_category == 'Dépense']

In [ ]:
entete = [k for k in data_list[0].keys()]
entete

In [ ]:
 model = ["accounting_category", "spending_item_name", "financial_institution", "type", "amount", "frequency", "recurrence_interval", "is_recurring", "due_date"]

In [ ]:
[budget_item.calculate_amount() for budget_item in [st.budget_set.filter(spending_theme__name="logement", owner__id=user.id) for st in SpendingTheme.objects.all()]]

In [ ]:
[budget_item.calculate_amount() for budget_item in[st.budget_set.filter(spending_theme__name="logement", owner__id=user.id) for st in SpendingTheme.objects.all() if st.budget_set.filter(spending_theme__name="logement", owner__id=user.id)][0]]

# Budget model

In [1]:
budget_items = Budget.objects.all()

In [2]:
for budget_item in budget_items:
    print(budget_item.spending_theme, budget_item.spending_item_name)

enfants Activité Enfants
None Allocation canadienne pour enfants
None Allocation famille QC
logement Apple iCloud
logement assurances
None BS
divertissement Cadeau
self-care Cheveux
None CLub TRI PAT
self-care Course Geneviève
enfants Dentiste Enfants
self-care Dentiste Genevieve
nourriture Épicerie
self-care Estheticienne
enfants frais de garde
enfants Frais scholaires
logement Hydro
logement Hypothèque
transport Immatriculations
transport imprévues
logement Internet
self-care Massage
divertissement Massages
divertissement Netflix
enfants Nourriture Cochons
None Paye
transport Permis de conduire
self-care Pharmacie
transport pneus
None Remboursement d'impôt
alchool et restaurant Restaurant
None Suppl. fournitures scolaires
logement Taxe municipales
logement Taxe scholaire
transport Transport (electricité / parking / metro-bus)
enfants Vetement Enfants
vêtements Vetement Genevieve
alchool et restaurant vie sociale
alchool et restaurant vin
transport voiture 50000 dans 5 ans
divertissem

In [4]:
merchants_qs = Merchant.objects.all()

In [ ]:
categories = CashFlowAccountingCategorie.objects.all()
for cat in categories:
    print(cat)

In [ ]:
spending_themes = SpendingTheme.objects.all()

for spending in spending_themes:
    print(spending)

In [ ]:
Merchant.objects.filter(categories)

In [11]:
epicerie = Budget.objects.get(spending_item_name="Épicerie")

for merchant in merchants_qs.order_by('name'):
    if merchant.categories.all():
        pass
    else:
        print(merchant)
        split_name = merchant.name.split()
        if "COSTCO" in split_name or "SUPER" in split_name or "ADONIS" in split_name or "IGA" in split_name or "MAXI" in split_name or "KIM" in split_name or "METRO" in split_name:
            merchant.categories.add(epicerie)
            merchant.save()
            print("&&&&&&&&&&", merchant)
        
   

 Achat / DOLLARAMA # 34
 Achat / LE RESTAURANT A
 Débit - effet divers
 Dépôt - Virement Interac
 Dépôt direct / BENEVA INC.
 Dépôt Mobile
 Retrait direct / Tangerine
 Retrait direct / Wealthsimple Investments Inc.
 Transfert de fonds inter I.F. / Tangerine
#414 SPORTS EXPERTS ANJOU QC
Achat / #468 SPORTS EXPERTS
Achat / DEPANNEUR DE LA PISTE
Achat / DOLLARAMA # 285
Achat / DOLLARAMA # 4
Achat / DOLLARAMA #1241
Achat / MARCHE INNOVATION
Achat / PATRICK MORIN 04
Achat / SUPER C PTE AUX TREMBL
&&&&&&&&&& Achat / SUPER C PTE AUX TREMBL
ALLO MON COCO PAT PTE AUX TREMBQC
Amazon.ca*KP8828J23 AMAZON.CA ON
Amazon.ca*SD6270PY3 AMAZON.CA ON
AMZ*PremiumpartsWhosal WWW.AMAZON.CAON
AMZN Mktp CA WWW.AMAZON.CAON
AMZN Mktp CA*0L0WZ4IS3 WWW.AMAZON.CAON
AMZN Mktp CA*156UA1NW3 WWW.AMAZON.CAON
AMZN Mktp CA*1F3HO5IV3 WWW.AMAZON.CAON
AMZN Mktp CA*4984K8R43 WWW.AMAZON.CAON
AMZN Mktp CA*4L5L22N53 WWW.AMAZON.CAON
AMZN Mktp CA*5002Y9IQ3 WWW.AMAZON.CAON
AMZN Mktp CA*AS0TR8WS3 WWW.AMAZON.CAON
AMZN Mktp CA*CO1SM78

In [12]:
transactions = Transaction.objects.all()

In [23]:
# get the merchant in eepicerie
epicerie_merchant = Merchant.objects.filter(categories=epicerie)

for transaction in transactions:
    if transaction.merchant in [x.name for x in epicerie_merchant]:
        transaction.category = epicerie
        # transaction.merchant_2 = epicerie_merchant.filter(name=transaction.merchant).first()
        transaction.save()
        print(transaction, transaction.merchant, epicerie_merchant.filter(name=transaction.merchant).first())

2023-04-25  Épicerie  25.55 SUPER C PAT 5963 POINTE AUX TRQC SUPER C PAT 5963 POINTE AUX TRQC
2023-04-25  Épicerie  41.39 MAXI #8987 * POINTE-AUX-TRQC MAXI #8987 * POINTE-AUX-TRQC
2023-04-29  Épicerie  29.33 ADONIS 21943 ANJOU ANJOU QC ADONIS 21943 ANJOU ANJOU QC
2023-04-29  Épicerie  23.54 SUPER C PAT 5963 POINTE AUX TRQC SUPER C PAT 5963 POINTE AUX TRQC
2023-05-01  Épicerie  4.59 MAXI #8987 * POINTE-AUX-TRQC MAXI #8987 * POINTE-AUX-TRQC
2023-05-02  Épicerie  4.99 METRO DE LA ROUSSELIER POINTE AUX TRQC METRO DE LA ROUSSELIER POINTE AUX TRQC
2023-05-04  Épicerie  31.09 SUPER C PAT 5963 POINTE AUX TRQC SUPER C PAT 5963 POINTE AUX TRQC
2023-05-05  Épicerie  24.57 METRO DE LA ROUSSELIER POINTE AUX TRQC METRO DE LA ROUSSELIER POINTE AUX TRQC
2023-05-06  Épicerie  14.5 MAXI & CIE REPENTIGNY REPENTIGNY QC MAXI & CIE REPENTIGNY REPENTIGNY QC
2023-05-07  Épicerie  40.28 METRO BEAUBIEN MONTREAL QC METRO BEAUBIEN MONTREAL QC
2023-05-07  Épicerie  7.99 IGA EXTRA 259 ST-LEONARD QC IGA EXTRA 259 ST

In [30]:
from investment.forms import CombinedForm

transactions_with_empty_categories_list = Transaction.objects.annotate(
                merchant_categories_count=Count('merchant_2__categories')).filter(
                    merchant_categories_count=0)
            
# transaction_form_to_review_list = [CombinedForm(instance=t) 
#                               for t in transactions_with_empty_categories_list]


for t in transactions_with_empty_categories_list:
    print(t.category, t.merchant_2)

Cadeau Variable 400.0 SP JULES & HENRI MONTREAL QC
Restaurant Variable 1000.0 MICROBRASSERIE LA MEMPH MAGOG QC
Restaurant Variable 1000.0 MCDONALD'S #18127 QPS ANGE-GARDIEN QC
Transport (electricité / parking / metro-bus) Variable 300.0 STM LAURIER DIN101 MONTREAL QC
Pharmacie Variable 200.0 PHARMACIE VINH THE JIMMY MONTREAL QC
Voyages Variable 7000.0 DOLLARAMA # 285 REPENTIGNY QC
Pharmacie Variable 200.0 JEAN COUTU 046 REPENTIGNY QC
Vetement Enfants Variable 300.0 OLDNAVY.COM 2944 505-462-0076 ON
Vetement Enfants Variable 300.0 OLDNAVY.COM 2944 505-462-0076 ON
Pharmacie Variable 200.0 JEAN COUTU 341 REPENTIGNY QC
Restaurant Variable 1000.0 LES ENFANTS TERRIBLES MONTREAL QC
vin Variable 120.0 SQ *STOCK AND ROW Bloomfield ON
Voyages Variable 7000.0 CITY OF KINGSTON-PARKING KINGSTON ON
Pharmacie Variable 200.0 PHARMACIE VINH THE JIMMY MONTREAL QC
Voyages Variable 7000.0 SEPAQ 1 800 665 6527 QUEBEC
Restaurant Variable 1000.0 MICROBRASSERIE COATICOOK COATICOOK
Voyages Variable 7000.0 DVERT

In [26]:
for transaction_form_to_review in transaction_form_to_review_list:
    print(transaction_form_to_review)

<tr>
    <th><label for="id_category">Category:</label></th>
    <td>
      
      <select name="category" id="id_category">
  <option value="">---------</option>

  <option value="827">Activité Enfants Variable 2000.0</option>

  <option value="838">Allocation canadienne pour enfants Fixe 7032.6</option>

  <option value="855">Allocation famille QC Fixe 200.0</option>

  <option value="852">Apple iCloud Fixe 1.6</option>

  <option value="806">assurances Fixe 166.15</option>

  <option value="862">BS Fixe 1050.0</option>

  <option value="822" selected>Cadeau Variable 400.0</option>

  <option value="814">Cheveux Variable 25.0</option>

  <option value="857">CLub TRI PAT Variable 300.0</option>

  <option value="854">Course Geneviève Variable 1000.0</option>

  <option value="861">Dentiste Enfants Fixe 320.0</option>

  <option value="860">Dentiste Genevieve Fixe 500.0</option>

  <option value="863">Épicerie Variable 150.0</option>

  <option value="813">Estheticienne Variable 80.0</